In [1]:
import npc_sessions
import numpy as np
import matplotlib.pyplot as plt

In [5]:
units = npc_sessions.get_units_electrodes('636766_2023-01-26')
units['structure_acronym']

0       root
1       root
2       root
3       root
4       root
        ... 
1962     MOs
1963     MOs
1964     ILA
1965     ILA
1966     ILA
Name: structure_acronym, Length: 1967, dtype: object

In [3]:
spike_times = npc_sessions.get_unit_spike_times_dict('636766_20230124', tuple(units['unit_name']))
spike_times

{'e0984cc5-af22-4d1c-90b9-6ae0bba4a590': array([   7.10708245,    8.07877977,    8.46007871, ..., 4885.70986089,
        4885.80416063, 4885.84059387]),
 '9800cb40-f4c3-4ad9-a0d2-8a61d71e71b3': array([   6.09881858,    6.14881844,    6.20101829, ..., 4888.40645344,
        4888.42342006, 4888.43082004]),
 '25232cd4-4a02-4e3e-a720-8270347e4269': array([   6.11018521,    6.5006508 ,    6.72405018, ..., 4886.05699327,
        4886.9017576 , 4888.16315411]),
 'c097894a-90a7-4ad4-83ea-07c1596a37da': array([   6.15515175,    6.19558497,    6.42145102, ..., 4888.68601933,
        4888.70568595, 4888.75691914]),
 'bb900b43-9c7e-4064-b87e-a495fb387546': array([   6.14661844,    6.34045124,    6.38928444, ..., 4888.43585336,
        4888.66038607, 4888.80635233]),
 '36be9a0a-4af3-4a31-a22c-de4c8ee021d2': array([   6.08645194,    6.09991857,    6.1271185 , ..., 4885.56259464,
        4885.59076122, 4888.03732113]),
 '1649860b-e395-4acb-a872-39a96b05ff7d': array([   6.18585167,    6.24391817,    6

In [5]:
len(units) == len(spike_times)

True

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import npc_sessions
import polars as pl

sessions = []
for s in npc_sessions.tracked:
    if not s.is_uploaded:
        continue
    try:
        _ = npc_sessions.get_units_electrodes(s.session)
    except (FileNotFoundError, TypeError) as exc:
        print(f'{s.session}: {exc!r}')
    else:
        print(f'{s.session}: units available')
        sessions.append(npc_sessions.Session(s.session))



session = next(s for s in sessions if s.id == '626791_2022-08-17')




stim = next(s for s in session.stim_data if 'DynamicRouting' in s)
trials = npc_sessions.DynamicRouting1(session.stim_data[stim], session.sync_data)
df = npc_sessions.get_units_electrodes_spike_times(session.id)

vis = df.filter(pl.col('structure_acronym').str.contains('VIS'))
aud = df.filter(pl.col('structure_acronym').str.starts_with('AUD'))

for spike_times in aud['spike_times']:
# spike_times = vis[random.randrange(len(vis))]['spike_times'][0]

    pad_start = .5
    pad_end = .5
    # align_on = 'response_window_start_time'
    align_on_time = 'stim_start_time'
    on = trials.to_dataframe().query('is_aud_stim')[align_on_time].values - pad_start
    off = pad_start + on + pad_end

    fig, ax = plt.subplots()
    ax.eventplot(
        [spike_times[a:b] - on[idx] - pad_start 
        for idx, (a, b)
        in enumerate(zip(np.searchsorted(spike_times, on), np.searchsorted(spike_times, off)))]
    )
    ax.axvline(0, color=[.8]*3, linestyle='--')
    ax.set(xlabel='time, s', ylabel='trials', xmargin=0, ymargin=0)
    ax2 = ax.secondary_xaxis('top')
    ax2.set(xticks=[0], xticklabels=[align_on_time]);
    plt.show()